In [1]:
import volteracamera.intrinsics.circles_calibration as cc
import volteracamera.intrinsics.screen_based_calibration as sc
import volteracamera.intrinsics.calibration_generation as cg
from volteracamera.analysis.transform import Transform
import transforms3d as tfd
import numpy as np
import random

#Generate base image.
image, points, pattern = cc.generate_symmetric_circle_grid()
width, height = sc.get_image_dimensions(image)
Ks = sc.get_camera_matrix(width, height)
#Generate the rvecs and tvecs.
random.seed(1)
transforms = [cg.generate_random_rvec_tvec() for _ in range(20)]
Hs = [sc.Homography.calculate_H(Ks, Transform(rotation = list(rvec), translation = list(tvec))) for rvec, tvec in transforms]
#generate the images on the screen.
images_on_screen = [sc.create_projected_image(image, rvec, tvec) for rvec, tvec in transforms]

screen_points = [ cc.analyze_calibration_image(image, pattern, display=False) for image in images_on_screen ]
measured_Hs = [ sc.Homography(points, image_points) for image_points in screen_points ]

for calc, meas in zip(Hs, measured_Hs):
    np.testing.assert_array_almost_equal (calc.H, meas.H)

AssertionError: 
Arrays are not almost equal to 6 decimals

(mismatch 100.0%)
 x: array([247.3421  ,  -2.800822,  99.640084,  -2.732986, 247.119618,
       125.326185,   0.      ,   0.      ,   0.362373])
 y: array([-7.509803e-01,  2.310719e-01,  6.241243e+02,  2.648826e-01,
       -6.972059e-01,  7.732455e+02,  5.647822e-04,  5.881925e-04,
        1.000000e+00])

# Intrinsics Calculation

This workbook summarizes the method used to calibrate the Voltera laser cameras intrinsics. The method involves generating images of a checkerboard/ball grid pattern on a screen, and then imaging the screen with the camera to be calibrated. The advantage of this method over the classis Zhang method is that is does not require moving the camera or target relative to each other, which is important given the narrow depth of field and high magnification of the Voltera camera.

The method is based on the paper:

> Tan L., Wang Y., Yu H., Zhu J., *"Automatic Camera Calibration Using Active Displays of a Virtual Pattern"*, **Sensor**, 17, 685, 2017

which builds on the classic Zhang paper for camera calibration

> Zhang Z., *"A Flexible New Technique for Camera Calibration"*, Microsoft Technical Report, MSR-TR-98-71, December 2, 1998


# Method

# Classical Calibration Method

To understand the calibration through a screen (the screen process), we need to understand the classical calibration process (the Zhang process).

Assume that we have a set of $n$ 3D points located on the $z=0$ plane. That plane is then imaged with a pinhole camera $m$ times (we'll ignore the distortion parameters for now). For each of the images (denoted by $j$), each is taken from a different positions, given by the $3\times3$ rotation matrix $R_j$ and $3\times1$ translation vector $t_j$. These transformations take the image points $ X_i = \left[ x_i, y_i, 0\right]^T$ ($i < n$) to their 2D projection $m_{ij} = \left[ u_{ij}, v_{ij}\right]^T$ through the following equation

$$
\begin{eqnarray}
\begin{bmatrix} m_{ij} \\ 1 \end{bmatrix} & \propto & K \begin{bmatrix} R_j & t_j \end{bmatrix} \begin{bmatrix} X_i \\ 1 \end{bmatrix}
\end {eqnarray}
$$

Where the K is the standard pinhole projection matrix:

$$
\begin{eqnarray}
K & = & \begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix} 
\end {eqnarray}
$$

Where $f_x$ and $f_y$ are the x and y focal lengths, and $c_x$, $c_y$ are the principle points (center of the image).
 
Since $z=0$ for the 3D points, the above can be reduced down to:

$$
\begin{eqnarray}
\begin{bmatrix} m_{ij} \\ 1 \end{bmatrix} & \propto & K \begin{bmatrix} r_{j1} & r_{j2} & t_j \end{bmatrix} \begin{bmatrix} x_i \\ y_i \\ 1 \end{bmatrix}
\end {eqnarray}
$$

We will simplify the above by defining the homography matrix for the $j$'th image as $H_j$:

$$
\begin{eqnarray}
H_j & = & K \begin{bmatrix} r_{j1} & r_{j2} & t_j \end{bmatrix} 
\end {eqnarray}
$$

Given 4 point correspondences between $X_i$ and $m_{ij}$, we can calculate the homography. 

## Constraints on the Homography

Given an image of a plane, the homography can be estimated (for instance using solvePNP methods). Using the above definition of $H_j$ and also using $H_j = \left[ h_{j1} h_{j2} h_{j3} \right]$, the above equation can be re-written:

$$
\begin{eqnarray}
\begin{bmatrix} h_{j1} & h_{j2} & h_{j3} \end{bmatrix} & = & \lambda K \begin{bmatrix} r_{j1} & r_{j2} & t_j \end{bmatrix} 
\end {eqnarray}
$$

Where the $\lambda$ is an arbitrary scalar. Using the orthonormality of the rotation matrix, we get the following contraints:

$$
\begin{eqnarray}
h^T_{j1} (K^{-1})^T K^{-1} h_{j2} & = & 0 \\
h^T_{j1} (K^{-1})^T K^{-1} h_{j1} - h^T_{j2} (K^{-1})^T K^{-1} h_{j2} & = & 0
\end {eqnarray}
$$

Derivation of this constraint is non-trivial. We'll take it as a given for now. It has a geometric interpretation/derivation using the absolute conic. Zhang covers it in some detail, but also check Multiple View Geometry for more info.

Simplifying above using $B = (K^{-1})^T K^{-1}$:

$$
\begin{eqnarray}
h^T_{j1} B h_{j2} & = & 0 \\
h^T_{j1} B h_{j1} - h^T_{j2} B h_{j2} & = & 0
\end {eqnarray}
$$

Where B is 3x3 symmetric, so have 6 components, but due to scale ambiguity, really only has 5. 

## Estimating Parameters

The detailed derivation is given in Zhang. 

$$
\begin{eqnarray}
B & = & \begin{bmatrix} B_{11} & B_{12} & B_{13} \\ B_{21} &  B_{22} & B_{23} \\ B_{31} &  B_{32} & B_{33} \end{bmatrix} \\
 & = & \begin{bmatrix} \frac{1}{f_x^2} & 0 & -\frac{c_x f_y}{f_x^2 f_y} \\
 0 & \frac{1}{f_y^2} & -\frac{c_y}{f_y^2}\\
 -\frac{c_x f_y}{f_x^2 f_y} & -\frac{c_y}{f_y^2} & \frac{\left(c_x f_y\right)^2}{f_x^2 f_y^2} + \frac{c_y^2}{f_y^2} + 1 \end{bmatrix}
\end {eqnarray}
$$

$B$ is symmetric so we can defind a 6D vector:

$$
\begin{eqnarray}
b & = & [ B_{11}, B_{12}, B_{22}, B_{13}, B_{23}, B_{33} ]^T
\end {eqnarray}
$$

Let's define the columns of $H$ as $h_i = \left[ h_{i1}, h_{i2}, h_{i3} \right]^T$, so that we have:

$$
\begin{eqnarray}
h_i^T B h_j & = & v_{ij}^T b
\end {eqnarray}
$$

where 

$$
\begin{eqnarray}
v_{ij} & = & \left[ h_{i1}h_{j1}, h_{i1}h_{j2} + h_{i2}h_{j1}, h_{i2}h_{j2}, h_{i3}h_{j1} + h_{i1}h_{j3}, h_{i3}h_{j2} + h_{i2}h_{j3}, h_{i3}h_{j3}\right]^T
\end{eqnarray}
$$

With the above equation, the two contraints on the homography can be re-written as:

$$
\begin{eqnarray}
\begin{bmatrix} v_{12}^T \\ (v_{11} - v_{22})^T\end{bmatrix} b & = & 0
\end{eqnarray}
$$

For $n$ images, we stack the various solutions to get $Vb = 0$, where $V$ is a $2n \times 6$ matrix.

So, to estimate the parameters of the camera calibration, we must find the eigenvectors of the matrix $V^TV$ with the smalles eigenvalues (After finding the homography using another method). Once B is estimated, we can compute al of the camera intrinsics parameters. 

Once $K$ is known, we can estimate the extrinsics parameters as:

$$
\begin {eqnarray}
r_1 & = & \lambda K^{-1}h_1 \\
r_2 & = & \lambda K^{-1}h_2 \\
r_3 & = & r1 \times r2 \\
t & = & \lambda K^{-1}h_3 \\
\lambda & = & 1 / \lvert A^{-1} h_1\rvert
\end{eqnarray}
$$

## Estimation of Radial Distortion

See Zhang page 8 equation 13.

## Computing Maximum Likelihood

The above estimation fails in the presence of noise, so use the above approximate solution to see the minimization of:

$$
\begin{eqnarray}
\sum_{j=1}^m \sum_{i=1}^n \lvert m_{ij} - p(X_i, K, R_j, t_j, d) \rvert
\end{eqnarray}
$$

Where $d$ are the lens distortion parameters and p is the projective function.

# Screen Based Method

The screen based method consists of projecting a simulated checkerboard onto a screen, which introduces it's own homography (known) followed by a image capture which further projects the image to the sensor surface.

Let the projection from screen to camera be $P = K \left[ R t \right]$ and the projection of the virtual pattern to the screen be $P^s_j = K^s \left[ R^s_j t^s_j \right]$ (for the j'th image).

The projection of the virtual 3D point and the 2D image point is given by:

$$
\begin{eqnarray}
\begin{bmatrix}m_{ij} \\ 1\end{bmatrix} & = &
\left[ I  0 \right] \begin{bmatrix} P \\ 0^T 1 \end{bmatrix}
\begin{bmatrix} P^s_j \\ 0^T  1 \end{bmatrix}
\begin{bmatrix} X_i \\  1 \end{bmatrix}
\end {eqnarray}
$$

Where $0$ is a $3\times1$ matrix and $I$ is a $3\times3$ identity matrix.

We'll start by considering the first projection (virtual to screen) following the logic from the classic method.

$$
\begin{eqnarray}
\begin{bmatrix} P^s_j \\ 0^T  1 \end{bmatrix}\begin{bmatrix} X_i \\  1 \end{bmatrix} & = &
\begin{bmatrix} K^s & 0 \\ 0^T & 1 \end{bmatrix} \begin{bmatrix} R^s_j & t^s_j \\ 0^T & 1 \end{bmatrix}\begin{bmatrix} X_i \\  1 \end{bmatrix} \\
&=& \begin{bmatrix} K^s & 0 \\ 0^T & 1 \end{bmatrix} \begin{bmatrix} r^s_{j1} & r^s_{j2} & t^s_j \\ 0 & 0 & 1 \end{bmatrix}\begin{bmatrix} X_i \\  1 \end{bmatrix} \\
&=& \begin{bmatrix} H^s_j \\ 0 1 \end{bmatrix} \begin{bmatrix} X_i \\  1 \end{bmatrix}
\end {eqnarray}
$$

Everything in $H^s_j$ (the screen homography for image j) is known. Using the above equations, we can rewrite the total projection function for the camera screen system as a single homography of the virtual x y point on the z=0 axis as:

$$
\begin{eqnarray}
\begin{bmatrix}m_{ij} \\ 1\end{bmatrix} & \propto &
H_j \begin{bmatrix} x_i \\ 1 \end{bmatrix} \\
H_j & \propto & K \begin{bmatrix} Rh^s_{j1} & Rh^s_{j2} & Rh^s_{j3}+t  \end{bmatrix}
\end {eqnarray}
$$ 

Where $H_j$ (the total homography) can be calculated in the same way as the standard Zhang method. 

The above equation can be used to generate a new set of contraints (through some method I don't totally understand), given by:

$$
\begin{eqnarray}
\lvert h^s_{j2} \rvert^2 h^T_{j1} Bh_{j1} - \lvert h^s_{j1} \rvert^2 h^T_{j2} Bh_{j2} & = & 0 \\
\left( h^{sT}_{j1}h^s_{j2}\right) h^T_{j1} B h_{j1} - \lvert h^s_{j1} \rvert^2 h^T_{j2} B h_{j1} & = & 0
\end {eqnarray}
$$ 

Where $B = \left( K^{-1} \right)^T K^{-1}$ is the only unknown, as before.

## Estimating Parameters

Following the logic above, we vectorize the symmetric matrix B and re-arrange the above contraints into equations of the form using the previously defined identies:

$$
\begin{eqnarray}
b & = & [ B_{11}, B_{12}, B_{22}, B_{13}, B_{23}, B_{33} ]^T \\
h_i^T B h_j & = & v_{ij}^T b \\
v_{ij} & = & \left[ h_{i1}h_{j1}, h_{i1}h_{j2} + h_{i2}h_{j1}, h_{i2}h_{j2}, h_{i3}h_{j1} + h_{i1}h_{j3}, h_{i3}h_{j2} + h_{i2}h_{j3}, h_{i3}h_{j3}\right]^T
\end {eqnarray}
$$

The difference in this case is in the form of the contraints. Using the above equations with the new constraints yields:

$$
\begin{eqnarray}
\begin{bmatrix} 
\left( \lvert h^s_{j2} \rvert^2 v_{11} - \lvert h^s_{j1} \rvert^2 v_{22} \right)^T \\ 
\left( \left( h^{sT}_{j1}h^s_{j2}\right) v_{11} -  \lvert h^s_{j1} \rvert^2 v_{21} \right)^T 
\end {bmatrix} b & = & 0
\end {eqnarray}
$$ 

Continue as with the original calibration method by solving for b and extracting the K parameters.

To estimate the rotation and translation parameters from the screen to the sensor, we stack the measured homography of the total image and the previously found K paramters and the known homography each image (virtual point to screen):

$$
\begin{eqnarray}
\underbrace {\begin{bmatrix} K^{-1} H_1 & \cdots & K^{-1} H_m \end{bmatrix}}_C & = & \begin{bmatrix} R & t\end{bmatrix} \underbrace {\begin{bmatrix} \begin{matrix} \mu_1 H^s_1 \\ 0 1 \end{matrix} & \cdots &\begin{matrix} \mu_m H^s_m \\ 0 1 \end{matrix}\end{bmatrix} }_D 
\end{eqnarray}
$$ 

Where $\mu_j = \lvert K^{-1}h_{j1} \rvert / \lvert h^s_{j1} \rvert $ is a scaling factor. The above equation can be solved with:

$$
\begin{eqnarray}
\begin{bmatrix} R & t \end{bmatrix} = CD^T\left(DD^T\right)^{-1}
\end{eqnarray}
$$

Once we have estimated the camera matrix and the rotation/translation from the screen to camera, we can begin to iteratively solve for these parameters given by minimizing:

$$
\begin{eqnarray}
\sum_j^m \sum_i^n \lvert m_{ij} - p(X_i, K^s, R_j^s, t_j^s, K, R, t, d)\rvert^2
\end {eqnarray}
$$

Where $K$, $R$ and $t$ are minimized using the above as starting parameters.

